In [1]:
import moviepy.editor
import os
import random
import math
import glob
from tqdm import tqdm
import numpy as np

def video_clip_split(dir_path):
    length = 180 # 3분 (180초)
    short_edge = 256

    labels = ["non-agree"]

    for label in labels:
        directory = os.path.join(dir_path, label)
        dst_directory = os.path.join(dir_path + "-after-3m", label)

        os.makedirs(dst_directory, exist_ok=True)

        # list of videos
        inputs = glob.glob(dir_path + f"\{label}\*")
        inputs.sort()

        for idx, name in enumerate(tqdm(inputs, desc=f"{label} data 3m clipping")):
            try:
                orig_file_name = os.path.basename(name)

                if os.path.exists(os.path.join(dst_directory, orig_file_name.split(".mp4")[0] + "-" + f'{1:03d}.mp4')):
                    continue
                
                # import to moviepy
                clip = moviepy.editor.VideoFileClip(name)

                # 강제로 256x256으로 만듬. 비율 달라지는건 상관 x
                clip = clip.resize((short_edge, short_edge))
            
                clip_start = []
                duration = math.floor(clip.duration)

                if duration < 180:  # 3분 미만 X
                    continue 
                elif duration < 210:  # 3분 ~ 3분 30초 이하 1개
                    step = 120
                    clips_num = 1 
                elif duration < 240:  # 3분 30초 ~ 4분 미만 2개
                    step = 90
                    clips_num = 2
                elif duration < 320:  # 4분 ~ 5분 20초 미만 3개
                    step = 80
                    clips_num = 3
                elif duration < 400:  # 5분 20초 ~ 6분 40초 미만 4개
                    step = 80
                    clips_num = 4
                elif duration < 450:  # 6분 40초 ~ 7분 30초 미만 5개
                    step = 80
                    clips_num = 5
                else:                 # 7분 30초 이상 5개
                    step = 90
                    clips_num = 5

                # for i in range (0, duration-length, step):
                #     clip_start.append(i)

                clip_start = np.linspace(0, duration-length, duration//step, endpoint=True).astype(int).tolist()
                clip_start = random.sample(clip_start, clips_num)
                clip_start.sort()
                clips = []

                for start in clip_start:
                    # cut a subclip
                    clips.append(clip.subclip(start, start+length))
                    
                    # clips.append(clip.subclip(40, -10))
                    # break

                for idx, c in enumerate(clips):
                    c.write_videofile(os.path.join(dst_directory, orig_file_name.split(".mp4")[0] + "-" + f'{idx+1:03d}.mp4'))
                
                clip.close()
            except:
                print("ERROR!!!!!", duration, clip_start, orig_file_name)
                return

base_directory = r'D:\Video-Dataset\2022-NSR-v4'

video_clip_split(base_directory)

non-agree data 3m clipping: 100%|██████████| 160/160 [00:00<00:00, 40108.09it/s]


In [ ]:
duration = 399
length = 180
step = 80

clip_start = np.linspace(0, duration-length, duration//step, endpoint=True).astype(int).tolist()
# clip_start = list(set(clip_start))
clip_start

### Video frame clip and save

In [ ]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

cap = cv2.VideoCapture(r'D:\Video-Dataset\2022-NSR-v3\새 폴더\1.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 가로 길이 가져오기 
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 세로 길이 가져오기
fps = 30

fcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X') # codec 설정
out = cv2.VideoWriter(r'D:\Video-Dataset\2022-NSR-v3\새 폴더\2.mp4', fcc, fps, (1050, 720), isColor=True)

while (cap.isOpened()):
    ret, frame = cap.read()

    if ret:
        frame = np.array(Image.fromarray(np.array(frame)).crop((0,0,1050,720)))
        out.write(frame)
    else:
        print("Complete write all frame!")
        break

cap.release()
out.release()